### TFF To CSV

In [5]:
# -*- coding: utf-8 -*-
import os
import cv2
import tensorflow as tf
import os
import fitz  # PyMuPDF


# -------- 1) Esquema de features del TFRecord --------
# Ajusta los nombres si tu TFRecord usa otros keys
FEATURE_SPEC = {
    "image/encoded":            tf.io.FixedLenFeature([], tf.string),
    "image/height":             tf.io.FixedLenFeature([], tf.int64),
    "image/width":              tf.io.FixedLenFeature([], tf.int64),
    "image/object/bbox/xmin":   tf.io.VarLenFeature(tf.float32),
    "image/object/bbox/ymin":   tf.io.VarLenFeature(tf.float32),
    "image/object/bbox/xmax":   tf.io.VarLenFeature(tf.float32),
    "image/object/bbox/ymax":   tf.io.VarLenFeature(tf.float32),
}


def _parse_example(raw):
    """Devuelve (img_rgb_np, bboxes_norm) donde bboxes_norm = Nx4 [xmin,ymin,xmax,ymax] en [0,1]."""
    ex = tf.io.parse_single_example(raw, FEATURE_SPEC)

    # Imagen
    img = tf.image.decode_image(ex["image/encoded"], channels=3)     # [H,W,3], tf.uint8
    h  = int(ex["image/height"].numpy())
    w  = int(ex["image/width"].numpy())
    img = tf.reshape(img, (h, w, 3)).numpy()                         # -> np.uint8 (RGB)

    # Cajas normalizadas (VarLen -> denso)
    xmin = tf.sparse.to_dense(ex["image/object/bbox/xmin"]).numpy()
    ymin = tf.sparse.to_dense(ex["image/object/bbox/ymin"]).numpy()
    xmax = tf.sparse.to_dense(ex["image/object/bbox/xmax"]).numpy()
    ymax = tf.sparse.to_dense(ex["image/object/bbox/ymax"]).numpy()

    if xmin.size == 0:
        bboxes = []
    else:
        bboxes = [[float(xmin[i]), float(ymin[i]), float(xmax[i]), float(ymax[i])]
                  for i in range(len(xmin))]
    return img, bboxes


# -------- 2) Función principal: extraer recortes --------
def extract_qr_from_split(split_name, tfrecord_path, output_base):
    """
    Lee un TFRecord y guarda cada QR detectado como PNG.
    split_name: etiqueta para nombrar archivos (p.ej. 'train', 'val', 'test')
    tfrecord_path: ruta al .tfrecord
    output_base: carpeta base donde escribir resultados
    """
    split_dir = os.path.join(output_base, split_name)
    os.makedirs(split_dir, exist_ok=True)

    ds = tf.data.TFRecordDataset(tfrecord_path)

    for idx, raw in enumerate(ds):
        img, boxes = _parse_example(raw)

        h, w = img.shape[:2]
        if not boxes:
            # Sin cajas en este ejemplo
            continue

        for i, (xn1, yn1, xn2, yn2) in enumerate(boxes):
            # Normalizado -> pixeles
            x1 = max(0, int(xn1 * w))
            y1 = max(0, int(yn1 * h))
            x2 = max(0, int(xn2 * w))
            y2 = max(0, int(yn2 * h))

            # Verificación de área válida
            if x2 <= x1 or y2 <= y1:
                print(f"[!] Caja inválida en {split_name} idx={idx}, box={i}: "
                      f"({x1},{y1})-({x2},{y2})")
                continue

            crop = img[y1:y2, x1:x2]   # RGB
            if crop.size == 0:
                print(f"[!] Recorte vacío en {split_name} idx={idx}, box={i}")
                continue

            # Guardar (cv2 espera BGR)
            out_path = os.path.join(split_dir, f"{split_name}_{idx}_{i}.png")
            cv2.imwrite(out_path, cv2.cvtColor(crop, cv2.COLOR_RGB2BGR))

    print(f"[✓] Extraídos {split_name} -> {split_dir}")


In [2]:
gral = r"C:\Users\ovill\OneDrive\Escritorio\QR SSD\qr\Detect QR.v1i.tfrecord"
tfrecord_paths = {
        "train": gral + "\\train\\QR-code.tfrecord",
        "val":   gral + "\\train\\QR-code.tfrecord",
        "test":  gral + "\\train\\QR-code.tfrecord",
    }

output_base = r"C:\Users\ovill\OneDrive\Escritorio\QR SSD\qr\Pruebas_QR"

os.makedirs(output_base, exist_ok=True)

for split, path in tfrecord_paths.items():
        extract_qr_from_split(split, path, output_base)

[✓] Extraídos train -> C:\Users\ovill\OneDrive\Escritorio\QR SSD\qr\Pruebas_QR\train
[✓] Extraídos val -> C:\Users\ovill\OneDrive\Escritorio\QR SSD\qr\Pruebas_QR\val
[✓] Extraídos test -> C:\Users\ovill\OneDrive\Escritorio\QR SSD\qr\Pruebas_QR\test


### PDF to IMG

In [6]:


def pdf_a_jpg(directorio_pdfs, output_dir_base, dpi=300):
    """
    Convierte todas las páginas de todos los PDFs en un directorio a imágenes JPG.

    Args:
        directorio_pdfs (str): Ruta de la carpeta que contiene los archivos PDF.
        output_dir_base (str): Carpeta donde se guardarán todas las imágenes.
        dpi (int): Resolución (en puntos por pulgada). 300 es buena calidad.

    Returns:
        str: Mensaje con el total de páginas convertidas.
    """
    os.makedirs(output_dir_base, exist_ok=True)

    # Lista los PDFs en el directorio
    pdf_files = sorted([f for f in os.listdir(directorio_pdfs) if f.lower().endswith(".pdf")])
    total_paginas = 0

    for idx, pdf_file in enumerate(pdf_files):
        pdf_path = os.path.join(directorio_pdfs, pdf_file)
        nombre_base = os.path.splitext(pdf_file)[0]
        doc = fitz.open(pdf_path)

        # Convierte cada página a imagen
        for i in range(len(doc)):
            page = doc.load_page(i)
            pix = page.get_pixmap(dpi=dpi)

            output_path = os.path.join(output_dir_base, f"{nombre_base}_{i+1:03d}.jpg")
            pix.save(output_path)
            total_paginas += 1

    return f"[✔] {total_paginas} páginas convertidas y guardadas en: {output_dir_base}"

In [7]:
directorio_pdfs = r"C:\Users\ovill\OneDrive\Escritorio\QR SSD\PDF"
output_dir_base = r"C:\Users\ovill\OneDrive\Escritorio\QR SSD\IMG"
print(pdf_a_jpg(directorio_pdfs, output_dir_base))

[✔] 2976 páginas convertidas y guardadas en: C:\Users\ovill\OneDrive\Escritorio\QR SSD\IMG
